In [ ]:
import transformers
import torch
from transformers import BitsAndBytesConfig
# model_id = "aaditya/OpenBioLLM-Llama3-8B"

gpu = 1
device = f"cuda:{gpu}" if torch.cuda.is_available() else "cpu"
nf4_config = BitsAndBytesConfig(
   load_in_4bit=True,
   bnb_4bit_quant_type="nf4",
   bnb_4bit_use_double_quant=True,
   bnb_4bit_compute_dtype=torch.bfloat16
)
model_id = "aaditya/OpenBioLLM-Llama3-70B"
# model_nf4 = transformers.LlamaForCausalLM.from_pretrained(model_id, load_in_8bit=True, device_map="auto")
model_nf4 = transformers.AutoModelForCausalLM.from_pretrained(model_id, 
                                                 quantization_config=nf4_config,
                                                 device_map={"": device})
# tokenizer = transformers.LlamaTokenizer.from_pretrained(model_id)
pipeline = transformers.pipeline(
    "text-generation",
    model= model_nf4, #model_id,
    # model_kwargs={"torch_dtype": torch.bfloat16},
    tokenizer=model_id,
    # device=device,
)

messages = [
    {"role": "system", "content": "You are an expert and experienced from the healthcare and biomedical domain with extensive medical knowledge and practical experience. Your name is OpenBioLLM, and your job is to annotate dictionary features and contexts learned by interpretability techniques. Please answer the below message."},
    {"role": "user", "content": "Hello?"},
]

prompt = pipeline.tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)

terminators = [
    pipeline.tokenizer.eos_token_id,
    pipeline.tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

outputs = pipeline(
    prompt,
    max_new_tokens=256,
    eos_token_id=terminators,
    do_sample=True,
    temperature=0.1,
    top_p=0.9,
)

print(outputs[0]["generated_text"][len(prompt):])

In [ ]:
import torch
import os
import pandas as pd
import re
import json
# Initialize the pipeline once

def generate_text(prompt, pipeline, max_new_tokens=256, temperature=0.00001, top_p=0.99):
    """
    Generate text using the specified prompt and parameters.

    Args:
        prompt (str): The input prompt for text generation.
        max_new_tokens (int, optional): The maximum number of new tokens to generate. Defaults to 256.
        temperature (float, optional): The temperature value for sampling. Defaults to 0.0.
        top_p (float, optional): The top-p value for sampling. Defaults to 0.9.

    Returns:
        str: The generated text.
    """


    messages = [
        {"role": "system", "content": "You are an expert and experienced from the healthcare and biomedical domain with extensive medical knowledge and practical experience. Your job is to help annotate specific tasks by looking for common patterns within text. Please answer the below message."},
        {"role": "user", "content": prompt},
    ]

    prompt = pipeline.tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )

    terminators = [
        pipeline.tokenizer.eos_token_id,
        pipeline.tokenizer.convert_tokens_to_ids("<|eot_id|>")
    ]


    outputs = pipeline(
        prompt,
        max_new_tokens=max_new_tokens,
        eos_token_id=terminators,
        do_sample=True,
        temperature=temperature,
        top_p=top_p,
    )

    return outputs[0]["generated_text"][len(prompt):]

def load_global_questions_df(baseline):
    path = f"experiments/dict_att/mimiciii_clean/dictionary/questions/global/human_{baseline}_global_interp_questions.csv"
    return pd.read_csv(path)

def create_global_question_prompt(contexts, summary, choice1, choice2):
    prompt = "Given these medically relevant token contexts:\n"
    prompt += contexts + "\n\n"
    prompt += "and summary:\n"
    prompt += summary 
    prompt += "\n\nWhich of the following choices A or B is most likely to be associated with the conditions described in the summary and more importantly the contexts?\n"
    prompt += "A\n " + choice1 + "\n\n"
    prompt += "B\n" + choice2 + "\n"
    # prompt += "Please respond with the letter and the set of medical conditions associated with each letter. \n\n"
    prompt += "Please respond only with the letter, a single letter. \n\n"
    # prompt += "For example, if you believe the answer is A, respond with 'A [medical condition 1, medical condition 2, ...]'.\n\n"
    prompt += "For example, if you believe the answer is A, respond with 'A'.\n\n"
    # prompt += "Also respond with how confident you are from 1 to 4, with 4 being the most confident.\n\n"
    return prompt

# def create_global_question_prompt(contexts, summary, choice1, choice2):
#     prompt = "Given these medically relevant token contexts:\n"
#     prompt += contexts + "\n\n"
#     prompt += "and summary:\n"
#     prompt += summary
#     prompt += "\n\nWhich of the following choices A or B of medical codes is most likely to be associated with the conditions described in the summary and more importantly the contexts?\n"
#     prompt += "A\n " + choice1 + "\n\n"
#     prompt += "B\n" + choice2 + "\n"
#     prompt += "Please provide a step-by-step reasoning for your answer, focusing more on the given contexts than the summary. Consider the relationship between the contexts and the choices. After your reasoning, respond with the letter.\n\n"
#     prompt += "For example, if you believe the answer is A after your reasoning, you would respond with:\n"
#     prompt += "Reasoning: [step-by-step reasoning]\n"
#     prompt += "Answer: A \n\n"
#     # prompt += "Also include your confidence level from 1 to 4, with 4 being the most confident.\n\n"
#     return prompt

def run_global_interpretability_experiment(df, pipeline):
    nCorrect = 0
    response_vector = []
    for i, row in df.iterrows():
        prompt = create_global_question_prompt(row["contexts"], row["summary"], row["choice1"], row["choice2"])
        print(prompt)
        response = generate_text(prompt, pipeline)
        
        # Improved regex to extract 'a' or 'b' from sentences or standalone
        match = re.search(r'\b([Aa]|[Bb])\b', response)
        if match:
            answer_letter = match.group(0).lower()
            if row["truth"] == row["choice1"]:
                if answer_letter == 'a':
                    nCorrect += 1
                    response_vector.append(1)
                else:
                    response_vector.append(0)
            else:
                if answer_letter == 'b':
                    nCorrect += 1
                    response_vector.append(1)
                else:
                    response_vector.append(0)
        else:
            response_vector.append(0)
        
        print("LLM response:")
        print(response)
        print("\n\n")
    
    print("acc:", nCorrect/len(df))
    print("nCorrect:")
    return response_vector

def load_global_questions_df_top(baseline, n=100):
    path = f"experiments/dict_att/mimiciii_clean/dictionary/questions/global/human_{baseline}_global_interp_questions_top{n}.csv"
    return pd.read_csv(path)

L1_df = load_global_questions_df_top("L1", n="all")
# L1_df =  load_global_questions_df("L1")
run_global_interpretability_experiment(L1_df, pipeline)


In [ ]:
relu_df = load_global_questions_df("RELU")
# relu_df = load_global_questions_df_top("RELU", n=500)
run_global_interpretability_experiment(relu_df, pipeline)

In [ ]:
import torch
import os
import pandas as pd
import re
import json
# Initialize the pipeline once

def generate_text(prompt, pipeline, max_new_tokens=256, temperature=0.00001, top_p=0.99):
    """
    Generate text using the specified prompt and parameters.

    Args:
        prompt (str): The input prompt for text generation.
        max_new_tokens (int, optional): The maximum number of new tokens to generate. Defaults to 256.
        temperature (float, optional): The temperature value for sampling. Defaults to 0.0.
        top_p (float, optional): The top-p value for sampling. Defaults to 0.9.

    Returns:
        str: The generated text.
    """


    messages = [
        {"role": "system", "content": "You are an expert and experienced from the healthcare and biomedical domain with extensive medical knowledge and practical experience. Your job is to help annotate specific tasks by looking for common patterns within text. Please answer the below message."},
        {"role": "user", "content": prompt},
    ]

    prompt = pipeline.tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )

    terminators = [
        pipeline.tokenizer.eos_token_id,
        pipeline.tokenizer.convert_tokens_to_ids("<|eot_id|>")
    ]


    outputs = pipeline(
        prompt,
        max_new_tokens=max_new_tokens,
        eos_token_id=terminators,
        do_sample=True,
        temperature=temperature,
        top_p=top_p,
    )

    return outputs[0]["generated_text"][len(prompt):]

def load_baseline_contexts_df(dataset, baseline):
    path = f"experiments/dict_att/{dataset}/dictionary/questions/sample_filtered_{baseline}_questions.csv"
    return pd.read_csv(path)

def create_description_question_human(contexts, topk=4):
    prompt = f""
    for context_string in contexts:
        prompt += " " + context_string+ "\n"
    prompt += f"\n Describe the common pattern observed among these 4 contexts above. Think step by step. If there are common medical trends observed, or specific anatomy repeatedly used, please highlight those. Please respond with 8 words or less about the specific medical theme.\n"
    return prompt

def create_identification_question_human(contexts, topk=4):
    prompt = ""
    for idx, context in enumerate(contexts):
        prompt += f"Context {idx}: " + context +  "\n"
    prompt += f"\nWhich context is different from the other contexts? Please reply with only the context number. \n"
    print(prompt)
    return prompt

dataset = "mimiciii_clean"
baseline = "RELU"
topk=4
df = load_baseline_contexts_df(dataset, baseline)
# maybe we can reconstruct the original contexts dictionary from the dataframe
responses = {}
feature_summaries = {}
n_yes = 0
for index, row in df.iterrows():
    f_id = row["fid"]
    answer = row["random_context"]
    og_contexts = []
    for i in range(0, topk + 1):
        if row[f"context{i}"] != answer:
            og_contexts.append(row[f"context{i}"])
    contexts = []
    for i in range(0, topk + 1):
        contexts.append(row[f"context{i}"])
    prompt = create_identification_question_human(contexts, topk=topk)
    response = generate_text(prompt, pipeline)
    match = re.search(r'\b\d+\b', response)
    if match: # only add to the number correct if match is found
        index_response = int(match.group(0)) 
        if contexts[index_response] == answer:
            n_yes +=1
            desc_question = create_description_question_human(og_contexts, topk=topk)
            feature_summaries[f_id] = generate_text(desc_question, pipeline)
        else:
            feature_summaries[f_id] = None
    else:
        feature_summaries[f_id] = None
        
print(f"Total Identification Accuracy: {n_yes} / {len(df)}")

# now we need to generate the feature summaries using the og contexts, and save them
path = f"experiments/dict_att/mimiciii_clean/dictionary/human_{baseline}_feature_summaries_top{topk}.json" 
with open(path, "w") as f:
    json.dump(feature_summaries, f, indent=4)

In [ ]:
import pandas as pd
import re
import json
import random
def create_yes_no_question(context_strings):
    """
    Create a prompt from the specified list of context strings.

    Args:
        context_strings (list): A list of context strings.

    Returns:
        str: The generated prompt.
    """
    prompt = f""
    for context_string in context_strings:
        prompt += "Context: " + " ".join(context_string) + "\n"
    prompt += f"\nIs there one explicit medical concept observed in all 4 contexts?\n"
    return prompt

def create_dict_questions_yes_no(contexts_dictionary, topk=4):
    questions = {}
    for f_id, context_dict in contexts_dictionary.items():
        contexts = context_dict["context_string"][:topk]
        prompt = create_yes_no_question(contexts)
        questions[f_id] = prompt
        print(prompt)
    return questions

def create_description_question(context_strings):
    prompt = f""
    for context_string in context_strings:
        prompt += " ".join(context_string) + "\n"
    prompt += f"\n Describe the common pattern observed among these 4 contexts above. Think step by step. If there are common medical trends observed, or specific anatomy repeatedly used, please highlight those. Please respond with 8 words or less about the specific medical theme. \n"
    return prompt

def create_dict_questions_desc(contexts_dictionary, yes_dict, topk=4):
    questions = {}
    for f_id, context_dict in contexts_dictionary.items():
        if yes_dict[f_id]: 
            contexts = context_dict["context_string"][:topk]
            prompt = create_description_question(contexts)
            questions[f_id] = prompt # only store the ones that the LLM originally identified.
        else:
            questions[f_id] = None # if the LLM did not identify a common pattern, then we use this as extra information
    return questions

# Auto Identify all dictionary contexts and compute important metrics like total yes / total questions 
# store them in a dictionary 
def get_yes_no_responses(questions, pipeline):
    responses = {}
    n_yes = 0
    for f_id, question in questions.items():
        response = generate_text(question, pipeline)
        print("DEBUG:", response)
        if "yes" in response: 
            n_yes +=1
            responses[f_id] = True
        else:
            responses[f_id] = False
        # print(response)
    return responses, n_yes

def get_description_responses(questions, pipeline):
    responses = {}
    for f_id, question in questions.items():
        if question:
            response = generate_text(question, pipeline)
            responses[f_id] = response
            # print(response)
    return responses

def extract_feature_summaries(description_responses):
    feature_summaries = {}
    pattern = r"The common pattern.*?(?:is|are)[\s'\"]*(.*?)(?:[.]|$)"
    for f_id, sentence in description_responses.items():
        match = re.search(pattern, sentence, re.IGNORECASE)
        if match:
            common_pattern = match.group(1)
            feature_summaries[f_id] = common_pattern
        else:
            feature_summaries[f_id] = None
    return feature_summaries

def get_random_context(fid, random_contexts):
    other_fids = list(random_contexts.keys())
    random_int = random.randint(0, len(other_fids)-1)
    while other_fids[random_int] == fid:
        random_int = random.randint(0, len(other_fids)-1)
    other_fid = other_fids[random_int]
    return random_contexts[other_fid]["context_string"][random.randint(0, len(random_contexts[other_fid]["context_string"])-1)]

def create_identification_experiment_questions(random_contexts, topk=4):
    questions = {}
    for f_id, context_dict in random_contexts.items():
        contexts = context_dict["context_string"][:topk].copy()
        random_context = get_random_context(f_id, random_contexts)
        contexts.append(random_context)
        random.shuffle(contexts)
        prompt = ""
        for idx, context in enumerate(contexts):
            prompt += f"Context {idx}: " + " ".join(context) + "\n"
        
        prompt += f"\nWhich context is different from the other contexts? Please reply with only the context number. \n"
        questions[f_id] = {"prompt": prompt, "truth": random_context, "contexts": contexts}
        print(prompt)
    return questions

def identification_test(rand_identification_prompts, pipeline):
    responses = {}
    n_yes = 0
    for f_id, question in rand_identification_prompts.items():
        prompt = question["prompt"]
        contexts = question["contexts"]
        answer = question["truth"]
        response = generate_text(prompt, pipeline)
        match = re.search(r'\b\d+\b', response)
        if match: # only add to the number correct if match is found
            index_response = int(match.group(0)) 
            if contexts[index_response] == answer:
                responses[f_id] = True
                n_yes +=1
            else:
                responses[f_id] = True
        else:
            responses[f_id] = False
    return responses, n_yes

def identification_pipeline(contexts_dictionary, pipeline, topk=4, path=None):
    # step 1 identify all the yeses
    y_n_questions = create_identification_experiment_questions(contexts_dictionary, topk=topk)
    yes_dict, n_yes = identification_test(y_n_questions, pipeline)
    desc_dict_questions = create_dict_questions_desc(contexts_dictionary, yes_dict, topk=topk)
    # step 2 identify all the descriptions
    desc_dict = get_description_responses(desc_dict_questions, pipeline)
    # step 3 extract feature summaries
    feature_summaries = desc_dict
    print(f"Total Identification Accuracy: {n_yes} / {len(yes_dict)}")
    if path:
        with open(path, "w") as f:
            json.dump(feature_summaries, f, indent=4)
    return feature_summaries, n_yes


In [ ]:
import pandas as pd
import re
def create_yes_no_question_string(fid,context_strings):
    """
    Create a prompt from the specified list of context strings.

    Args:
        context_strings (list): A list of context strings.

    Returns:
        str: The generated prompt.
    """
    # prompt = "Here is a set of contexts below. Please read them carefully and answer the question at the end. \n\n"
    prompt = ""
    # prompt = "Here are some examples responses to the below question. \n \"Yes, the medical concept or pattern is \'blood transfusions\'.\" \n \"Yes, the medical concept or pattern is \'wound infection\'.\" \n We have 4 contexts below. Please read them carefully and answer the question at the end. \n\n"
    for context_string in context_strings:
        prompt += context_string + "\n"
    prompt += f"\nIs there one common theme observed among these 4 contexts above? Respond with only one word yes or no.\n"
    # prompt += "Please respond with the following format:\n"
    # prompt += "Answer: <yes or no>\n"
    # prompt += "Common Pattern: <common pattern description>\n\n"
    return prompt

def create_description_question_string(context_strings):
    prompt = ""
    for context_string in context_strings:
        prompt += context_string + "\n"
    prompt += f"\n Describe the common pattern observed among these 4 contexts above. Please respond with 6 words or less. \n"
    return prompt 

def create_list_of_questions_description_df(contexts_df, yes_dict):
    questions = {}
    for index, context_dict in contexts_df.iterrows():
        f_id = context_dict["f_id"]
        contexts = [context_dict["context1"], context_dict["context2"], context_dict["context3"], context_dict["context4"]]
        if f_id in yes_dict:
            prompt = create_description_question_string(contexts)
            questions[f_id] = prompt # only store the ones that the LLM originally identified.
        print(prompt)
    return questions

def extract_feature_summaries(description_responses):
    feature_summaries = {}
    pattern = r"The common pattern.*?(?:is|are)[\s'\"]*(.*?)(?:[.]|$)"
    for f_id, sentence in description_responses.items():
        match = re.search(pattern, sentence, re.IGNORECASE)
        if match:
            common_pattern = match.group(1)
            feature_summaries[f_id] = common_pattern
        else:
            feature_summaries[f_id] = None
    return feature_summaries
# columns f_id,question1,question2,context1,context2,context3,context4
def create_list_of_questions_yes_no_df(contexts_df):
    questions = {}
    for index, context_dict in contexts_df.iterrows():
        f_id = context_dict["f_id"]
        contexts = [context_dict["context1"], context_dict["context2"], context_dict["context3"], context_dict["context4"]]
        prompt = create_yes_no_question_string(f_id, contexts)
        questions[f_id] = prompt 
        print(prompt)
    return questions
contexts_df = pd.read_csv("experiments/dict_att/mimiciii_clean/dictionary/sampled_filtered_contexts.csv")

In [ ]:
questions = create_list_of_questions_yes_no_df(contexts_df)

In [ ]:
def get_yes_no_responses(questions, pipeline):
    responses = {}
    for f_id, question in questions.items():
        response = generate_text(question, pipeline)
        if "yes" in response: 
            responses[f_id] = True
        else:
            responses[f_id] = False
        print(response)
    return responses


responses = {}
for f_id, question in questions.items():
    response = generate_text(question, pipeline)
    if "yes" in response: 
        responses[f_id] = True
    else:
        responses[f_id] = False
    print(response)
# def get_common_patterns(responses)

In [ ]:
description_questions = create_list_of_questions_description_df(contexts_df, responses)

In [ ]:
description_responses = {}
for f_id, question in description_questions.items():
    response = generate_text(question, pipeline)
    description_responses[f_id] = response
    print(response)

In [ ]:
import re

sentences = description_responses.values()

pattern = r"The common pattern.*?(?:is|are)[\s'\"]*(.*?)(?:[.]|$)"

for sentence in sentences:
    match = re.search(pattern, sentence, re.IGNORECASE)
    if match:
        common_pattern = match.group(1)
        print("Common Pattern:", common_pattern)
    else:
        print("No common pattern found.")

# Baselines

In [ ]:
import os
def load_baseline_contexts(dataset, batch_size, n_batches, baseline):
    dir = os.path.join("experiments", "dict_att", dataset, "dictionary", "baseline_contexts")
    dir_path = os.path.join(dir, f"bdctxt_bs_{batch_size}_nb_{n_batches}_{baseline}.pt")
    return torch.load(dir_path)

dataset = "mimiciii_clean"
random_contexts = load_baseline_contexts(dataset, 2, 10, "RANDOM")
relu_contexts = load_baseline_contexts(dataset, 2, 10, "RELU")

In [ ]:
import random
def get_random_context(fid, random_contexts):
    other_fids = list(random_contexts.keys())
    random_int = random.randint(0, len(other_fids)-1)
    while other_fids[random_int] == fid:
        random_int = random.randint(0, len(other_fids)-1)
    other_fid = other_fids[random_int]
    return random_contexts[other_fid]["context_string"][random.randint(0, len(random_contexts[other_fid]["context_string"])-1)]

def create_identification_experiment_questions(random_contexts, topk=4):
    questions = {}
    for f_id, context_dict in random_contexts.items():
        contexts = context_dict["context_string"][:topk].copy()
        random_context = get_random_context(f_id, random_contexts)
        contexts.append(random_context)
        random.shuffle(contexts)
        prompt = ""
        for idx, context in enumerate(contexts):
            prompt += f"Context {idx}: " + " ".join(context) + "\n"
        
        prompt += f"\nWhich context is different from the other contexts? Please reply with only the context number. \n"
        questions[f_id] = {"prompt": prompt, "truth": random_context, "contexts": contexts}
        print(prompt)
    return questions

In [ ]:
def load_filtered_contexts(dataset, batch_size, n_batches):
    dir = os.path.join("experiments", "dict_att", dataset, "dictionary", "filtered_contexts")
    dir_path = os.path.join(dir, f"fdctxt_bs_{batch_size}_nb_{n_batches}.pt")
    return torch.load(dir_path)

filtered_contexts = load_filtered_contexts(dataset, 64, 100)

In [ ]:
subset_size = 500
subset_keys = random.sample(list(random_contexts.keys()), subset_size)
subset_rand_ctxt ={k: random_contexts[k] for k in subset_keys}

subset_keys = random.sample(list(filtered_contexts.keys()), subset_size)
subset_dict_ctxt ={k: filtered_contexts[k] for k in subset_keys}

subset_keys = random.sample(list(relu_contexts.keys()), subset_size)
subset_relu_prompts ={k: relu_contexts[k] for k in subset_keys}

In [ ]:
subset_dict_summaries, dict_yes = identification_pipeline(subset_dict_ctxt, pipeline, topk=4, path="experiments/dict_att/mimiciii_clean/dictionary/test_dictionary_summaries.json")

In [ ]:
def load_filtered_contexts(dataset, batch_size, n_batches):
    dir = os.path.join("experiments", "dict_att", dataset, "dictionary", "filtered_contexts")
    dir_path = os.path.join(dir, f"fdctxt_bs_{batch_size}_nb_{n_batches}.pt")
    return torch.load(dir_path)

import pickle

def load_pickle(file_path):
    """
    Loads a Python object from a pickle file.

    Args:
        file_path (str): The path to the pickle file.

    Returns:
        The Python object loaded from the file.
    """
    try:
        with open(file_path, 'rb') as f:
            obj = pickle.load(f)
        return obj
    except FileNotFoundError:
        print(f"Error: The file '{file_path}' does not exist.")
    except (pickle.UnpicklingError, EOFError):
        print(f"Error: The file '{file_path}' is not a valid pickle file.")
    except Exception as e:
        print(f"Error: Failed to load the pickle file '{file_path}'. Error: {e}")
        raise

def load_filtered_pkl_contexts(dataset, batch_size, n_batches):
    dir = os.path.join("experiments", "dict_att", dataset, "dictionary", "pkl_contexts")
    dir_path = os.path.join(dir, f"pdctxt_bs_{batch_size}_nb_{n_batches}.pt")
    return torch.load(dir_path)

def restore_dict(data_dict):
    for key, value in data_dict.items():
        if isinstance(value, dict):
            data_dict[key] = restore_dict(value)
        elif isinstance(value, list):
            new_list = []
            for item in value:
                if isinstance(item, list):
                    if all(isinstance(x, float) for x in item):
                        new_list.append(torch.tensor(item, dtype=torch.float32))
                    elif all(isinstance(x, int) for x in item):
                        new_list.append(torch.tensor(item, dtype=torch.int64))
                    else:
                        new_list.append(item)
                else:
                    new_list.append(item)
            data_dict[key] = new_list
        elif isinstance(value, (float, int)):
            data_dict[key] = torch.tensor([value], dtype=torch.float32 if isinstance(value, float) else torch.int64)
    return data_dict

dataset="mimiciii_clean"
global_ctxts = restore_dict(load_pickle("data.pkl"))

In [ ]:
identification_pipeline(global_ctxts, pipeline, topk=4, path="experiments/dict_att/mimiciii_clean/dictionary/test_global_summaries.json")

In [ ]:
rand_summaries, rand_yes = identification_pipeline(subset_rand_ctxt, pipeline, topk=4, path="experiments/dict_att/mimiciii_clean/dictionary/test_random_summaries.json")

In [ ]:
rand_identification_prompts = create_identification_experiment_questions(random_contexts, topk=4)
dictionary_identification_prompts = create_identification_experiment_questions(filtered_contexts, topk=4)
relu_identification_prompts = create_identification_experiment_questions(relu_contexts, topk=4)

In [ ]:
print(len(rand_identification_prompts))
print(len(dictionary_identification_prompts))
subset_size = 100
subset_keys = random.sample(list(rand_identification_prompts.keys()), subset_size)
subset_rand_prompts ={k: rand_identification_prompts[k] for k in subset_keys}

subset_keys = random.sample(list(dictionary_identification_prompts.keys()), subset_size)
subset_dict_prompts ={k: dictionary_identification_prompts[k] for k in subset_keys}

subset_keys = random.sample(list(relu_identification_prompts.keys()), subset_size)
subset_relu_prompts ={k: relu_identification_prompts[k] for k in subset_keys}

In [ ]:
subset_keys = random.sample(list(relu_identification_prompts.keys()), subset_size)
subset_relu_prompts ={k: relu_identification_prompts[k] for k in subset_keys}

In [ ]:
def pretty_print_tokens(token_lists):
    for token_list in token_lists:
        print(" ".join(token_list))

def identification_test(rand_identification_prompts, pipeline):
    responses = {}
    for f_id, question in rand_identification_prompts.items():
        prompt = question["prompt"]
        contexts = question["contexts"]
        answer = question["truth"]
        response = generate_text(prompt, pipeline)
        match = re.search(r'\b\d+\b', response)
        if match: # only add to the number correct if match is found
            index_response = int(match.group(0)) 
            if contexts[index_response] == answer:
                responses[f_id] = True
            else:
                responses[f_id] = False
        else:
            responses[f_id] = False
    return responses
    # print("LLM Correctly Identified?", responses[f_id])
    # print("Prompt:")
    # print(prompt)
    # print("LLM RESPONSE:")
    # print(response)
    # print("Correct Answer:")
    # print(answer)
    # print("Extracted Index:", index_response)
    # print("Contexts:")
    # pretty_print_tokens(contexts)


In [ ]:
rand_yes_no = identification_test(subset_rand_prompts, pipeline)

In [ ]:
dict_yes_no = identification_test(subset_dict_prompts, pipeline)

In [ ]:
relu_yes_no = identification_test(subset_relu_prompts, pipeline)

In [ ]:
print("Random Identification Accuracy:", sum(rand_yes_no.values())/len(rand_yes_no))
print("Dictionary Identification Accuracy:", sum(dict_yes_no.values())/len(dict_yes_no))
print("ReLU Identification Accuracy:", sum(relu_yes_no.values())/len(relu_yes_no))

In [ ]:
rand_y_n_questions = create_dict_questions_yes_no(random_contexts, topk=4)


In [ ]:
yes_dict, n_yes = get_yes_no_responses(rand_y_n_questions, pipeline)

In [ ]:
identification_pipeline(random_contexts, pipeline, topk=4, path="experiments/dict_att/mimiciii_clean/dictionary/feature_summaries_random.json")

In [ ]:
def identification_pipeline(contexts_dictionary, pipeline, topk=4, path=None):
    # step 1 identify all the yeses
    y_n_questions = create_dict_questions_yes_no(contexts_dictionary, topk=topk)
    yes_dict, n_yes = get_yes_no_responses(y_n_questions, pipeline)
    desc_dict_questions = create_dict_questions_desc(contexts_dictionary, yes_dict, topk=topk)
    # step 2 identify all the descriptions
    desc_dict = get_description_responses(desc_dict_questions, pipeline)
    # step 3 extract feature summaries
    feature_summaries = extract_feature_summaries(desc_dict)
    print(f"Total Yes: {n_yes} / {len(yes_dict)}")
    if path:
        with open(path, "w") as f:
            json.dump(feature_summaries, f, indent=4)
    return feature_summaries, n_yes

In [ ]:
contexts = [['found',
  ' to',
  ' have',
  ' a',
  ' subdural',
  ' hematoma',
  ' left',
  ' humerus',
  ' and',
  ' left'],
 [' l',
  'f',
  ' chief',
  ' complaint',
  ' subdural',
  ' hematoma',
  ' major',
  ' surgical',
  ' or',
  ' invasive'],
 [' over',
  ' year',
  ' old',
  ' patient',
  ' fell',
  ' from',
  ' standing',
  ' she',
  ' was',
  ' evaluated',
  ' at'],
 [' year',
  ' old',
  ' woman',
  ' who',
  ' fell',
  ' three',
  ' to',
  ' seven',
  ' days']]
question = create_description_question(contexts)
response = generate_text(question, pipeline)
print(response)